### Import block

In [1]:
import os
import pandas as pd
import datetime as dt
import numpy as np
import altair as alt
import geopy.distance as gpx
import geopandas as gpd
from scipy.sparse import coo_matrix
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

### Function definitions

In [2]:
def precip_indexer(t_lat, t_lon, t_time, t_date, t_year):
    '''
    Takes a desired location, date, time, and year; and returns the location of that row in the corresponding
    precipation DataFrame. Assumes that data is stored in a certain predictably sorted order.
    '''
    base_dt = dt.datetime.strptime(str(int(t_year)) + '/01/01','%Y/%m/%d').date()
    lat_idx = ((t_lat + 34) / .5)
    lon_idx = ((t_lon + 73.75) / 0.625)
    time_idx = (t_time)
    date_idx = (t_date - base_dt).days
    
    return int((date_idx * 24 * 64 * 81) + (time_idx * 64 * 81) + (lon_idx * 81) + lat_idx)

In [3]:
def one_hot_with_defo_precip_stack(px_combined,
                            defo_gpd,
                            precip_data,
                            centroid_lat = 0,
                            centroid_lon = 0,
                            centroid_date = dt.datetime.strptime('2017/01/01','%Y/%m/%d'),
                            span = 15,
                            days_back = 9,
                            days_forward = 10):
    
    min_date = centroid_date - dt.timedelta(days = days_back)
    max_date = centroid_date + dt.timedelta(days = days_forward)
    t_year = centroid_date.year

    # Calculate location if we move 375 meters North and use to get grid steps.
    lat_375 = gpx.distance(kilometers=0.375).destination((centroid_lat, centroid_lon), bearing=0).latitude
    lat_step = abs(centroid_lat - lat_375)

    # Calculate location if we move 375 meters East and use to get grid steps.
    lon_375 = gpx.distance(kilometers=0.375).destination((centroid_lat, centroid_lon), bearing=90).longitude
    lon_step = abs(centroid_lon - lon_375)

    # Get bounding box of centroid and span pixels in all directions.
    min_lat = centroid_lat - (lat_step * (span + 1))
    max_lat = centroid_lat + (lat_step * span)
    min_lon = centroid_lon - (lon_step * (span + 1))
    max_lon = centroid_lon + (lon_step * span)

    # Filter data down to the range we want to plot
    t_df = px_combined[(px_combined['Date'] >= min_date) & (px_combined['Date'] <= max_date)]
    t_df = t_df[(t_df['Lat'] >= min_lat) & (t_df['Lat'] <= max_lat) & (t_df['Lon'] >= min_lon) & (t_df['Lon'] <= max_lon)]

    # Calculate pixel locations in the new grid.
    t_df['delta_lat'] = t_df['Lat'] - centroid_lat
    t_df['delta_lon'] = t_df['Lon'] - centroid_lon
    t_df['lat_grid'] = t_df['delta_lat'] // lat_step
    t_df['lon_grid'] = t_df['delta_lon'] // lon_step

    # Express date difference
    t_df['delta_day'] = (t_df['Date'] - centroid_date).dt.days

    # Convert grids to not have negative locations.
    t_df['lat_grid'] = t_df['lat_grid'] + span + 1
    t_df['lon_grid'] = t_df['lon_grid'] + span + 1

    ### Begin added code for deforestation processing.

    # Limit deforestation polygons to those that intersect with centroid grid.
    centroid_defo = defo_gpd[(defo_gpd['x_min'] >= min_lon) & (defo_gpd['x_max'] <= max_lon) &
                   (defo_gpd['y_min'] >= min_lat) & (defo_gpd['y_max'] <= max_lat) &
                   (defo_gpd['x_min'] + defo_gpd['x_min'] + defo_gpd['x_min'] + defo_gpd['x_min'] != 0) &
                   (defo_gpd['Date'] <= min_date)]

    if len(centroid_defo) > 0:
        # Build grid.
        lats = [min_lat + lat_step * x for x in range(0,span * 2 + 2)]
        lons = [min_lon + lon_step * x for x in range(0,span * 2 + 2)]
        cs = [c for c in range(0,span * 2 + 2)]
        grid = list(itertools.product(lons, lats))
        grid_xy = list(itertools.product(cs, cs))
        grid_points = gpd.points_from_xy([x[0] for x in grid], [y[1] for y in grid])
        grid_gdf = gpd.GeoDataFrame(grid_points)
        grid_gdf.rename(mapper={0:'geometry'}, axis=1, inplace = True)
        grid_gdf['x'] = [c[0] for c in grid_xy]
        grid_gdf['y'] = [c[1] for c in grid_xy]
        grid_gdf = grid_gdf.set_geometry('geometry')
        grid_gdf.crs = 'epsg:4674'

        # Calculate deforested gridpoints.
        grid_gdf['deforested'] = np.minimum(pd.concat(
            [grid_gdf.within(centroid_defo.iloc[x].geometry.buffer(0)) for x in range(0,len(centroid_defo))]
            , axis=1).sum(1),1)

        # Convert to numpy format.
        dm = coo_matrix((grid_gdf['deforested'],(grid_gdf['x'].astype(int),grid_gdf['y'].astype(int)))).toarray()
    else:
        dm = np.zeros(((2 * span + 2),(2 * span + 2)))

    ### End added code for deforestaion processing.
    
    ### Begin added code for precip processing.
    w_lat_min = centroid_lat // 0.5 * 0.5
    w_lat_max = w_lat_min + 0.5
    w_lon_min = centroid_lon // 0.625 * 0.625
    w_lon_max = w_lon_min + 0.625
    wt = []
    
    for x in range(-days_back, 1):
        t_date = (centroid_date + dt.timedelta(days = x)).date()
        
        w1 = np.concatenate([precip_data.iloc[precip_indexer(w_lat_min, w_lon_min, x, t_date, t_year)].values.flatten() for x in range(0,24)])
        w2 = np.concatenate([precip_data.iloc[precip_indexer(w_lat_max, w_lon_min, x, t_date, t_year)].values.flatten() for x in range(0,24)])
        w3 = np.concatenate([precip_data.iloc[precip_indexer(w_lat_min, w_lon_max, x, t_date, t_year)].values.flatten() for x in range(0,24)])
        w4 = np.concatenate([precip_data.iloc[precip_indexer(w_lat_max, w_lon_max, x, t_date, t_year)].values.flatten() for x in range(0,24)])

        wt.append(np.mean([w1,w2,w3,w4], axis = 0))
        
    wt = np.stack(wt)
    ### End added code for precip processing.

    # Build an array for each day in our date range then stack them. One for the inputs and one for the outputs.
    input_stack = []
    for x in range(-days_back, 1):
        s_df = t_df[t_df['Date'] == (centroid_date + dt.timedelta(days = x))]
        if len(s_df) > 0:
            cm = coo_matrix((s_df['value'],(s_df['lon_grid'].astype(int),s_df['lat_grid'].astype(int)))).toarray()
            cm = np.pad(cm, pad_width=((0, (2 * span + 2) - cm.shape[0]),(0,(2 * span + 2) - cm.shape[1])), mode = 'constant')
        else:
            cm = np.zeros(((2 * span + 2),(2 * span + 2)))
        input_stack.append(cm)
    # Stack fires/defo.
    input_stack = np.stack([np.stack(input_stack), np.stack([dm]*10)], axis = -1)
    
    output_stack = []
    for x in range(1, days_forward + 1):
        s_df = t_df[t_df['Date'] == (centroid_date + dt.timedelta(days = x))]
        if len(s_df) > 0:
            cm = coo_matrix((s_df['value'],(s_df['lon_grid'].astype(int),s_df['lat_grid'].astype(int)))).toarray()
            cm = np.pad(cm, pad_width=((0, (2 * span + 2) - cm.shape[0]),(0,(2 * span + 2) - cm.shape[1])), mode = 'constant')
        else:
            cm = np.zeros(((2 * span + 2),(2 * span + 2)))
        output_stack.append(np.stack(cm))
    output_stack = np.stack(output_stack)
        
        
    return input_stack, wt, output_stack

In [4]:
def one_hot_with_defo_precip_stack_and_save(
                 centroid_df,
                 defo_gpd,
                 precip_data,
                 px_combined,
                 meta_folder = '/data/',
                 feature_folder_2d = '/data/features_2d/',
                 feature_folder_1d = '/data/features_1d/',
                 label_folder = '/data/labels/',
                 span = 15,
                 days_back = 10,
                 days_forward = 10):
    
    meta_df = centroid_df.copy()
    meta_df['features_2d'] = feature_folder_2d + meta_df.index.astype(str) + '.npy'
    meta_df['features_1d'] = feature_folder_1d + meta_df.index.astype(str) + '.npy'
    meta_df['labels'] = label_folder + meta_df.index.astype(str) + '.npy'
    
    for x in range(0,len(centroid_df)):
        
        t_index = centroid_df.index[x].astype(str)
        centroid_lat = centroid_df.iloc[x].Lat
        centroid_lon = centroid_df.iloc[x].Lon
        centroid_date = centroid_df.iloc[x].Date
        t_year = centroid_date.year
        
        min_date = centroid_date - dt.timedelta(days = days_back)
        max_date = centroid_date + dt.timedelta(days = days_forward)

        # Calculate location if we move 375 meters North and use to get grid steps.
        lat_375 = gpx.distance(kilometers=0.375).destination((centroid_lat, centroid_lon), bearing=0).latitude
        lat_step = abs(centroid_lat - lat_375)

        # Calculate location if we move 375 meters East and use to get grid steps.
        lon_375 = gpx.distance(kilometers=0.375).destination((centroid_lat, centroid_lon), bearing=90).longitude
        lon_step = abs(centroid_lon - lon_375)

        # Get bounding box of centroid and span pixels in all directions.
        min_lat = centroid_lat - (lat_step * (span + 1))
        max_lat = centroid_lat + (lat_step * span)
        min_lon = centroid_lon - (lon_step * (span + 1))
        max_lon = centroid_lon + (lon_step * span)
        
        # Filter data down to the range we want to plot
        t_df = px_combined[(px_combined['Date'] >= min_date) & (px_combined['Date'] <= max_date)]
        t_df = t_df[(t_df['Lat'] >= min_lat) & (t_df['Lat'] <= max_lat) & (t_df['Lon'] >= min_lon) & (t_df['Lon'] <= max_lon)]

        # Calculate pixel locations in the new grid.
        t_df['delta_lat'] = t_df['Lat'] - centroid_lat
        t_df['delta_lon'] = t_df['Lon'] - centroid_lon
        t_df['lat_grid'] = t_df['delta_lat'] // lat_step
        t_df['lon_grid'] = t_df['delta_lon'] // lon_step

        # Express date difference
        t_df['delta_day'] = (t_df['Date'] - centroid_date).dt.days

        # Convert grids to not have negative locations.
        t_df['lat_grid'] = t_df['lat_grid'] + span + 1
        t_df['lon_grid'] = t_df['lon_grid'] + span + 1
        
        ### Begin added code for deforestation processing.
        
        # Limit deforestation polygons to those that intersect with centroid grid.
        centroid_defo = defo_gpd[(defo_gpd['x_min'] >= min_lon) & (defo_gpd['x_max'] <= max_lon) &
                       (defo_gpd['y_min'] >= min_lat) & (defo_gpd['y_max'] <= max_lat) &
                       (defo_gpd['x_min'] + defo_gpd['x_min'] + defo_gpd['x_min'] + defo_gpd['x_min'] != 0) &
                       (defo_gpd['Date'] <= min_date)]
        
        if len(centroid_defo) > 0:
            # Build grid.
            lats = [min_lat + lat_step * x for x in range(0,span * 2 + 2)]
            lons = [min_lon + lon_step * x for x in range(0,span * 2 + 2)]
            cs = [c for c in range(0,span * 2 + 2)]
            grid = list(itertools.product(lons, lats))
            grid_xy = list(itertools.product(cs, cs))
            grid_points = gpd.points_from_xy([x[0] for x in grid], [y[1] for y in grid])
            grid_gdf = gpd.GeoDataFrame(grid_points)
            grid_gdf.rename(mapper={0:'geometry'}, axis=1, inplace = True)
            grid_gdf['x'] = [c[0] for c in grid_xy]
            grid_gdf['y'] = [c[1] for c in grid_xy]
            grid_gdf = grid_gdf.set_geometry('geometry')
            grid_gdf.crs = 'epsg:4674'

            # Calculate deforested gridpoints.
            grid_gdf['deforested'] = np.minimum(pd.concat(
                [grid_gdf.within(centroid_defo.iloc[x].geometry.buffer(0)) for x in range(0,len(centroid_defo))]
                , axis=1).sum(1),1)

            # Convert to numpy format.
            dm = coo_matrix((grid_gdf['deforested'],(grid_gdf['x'].astype(int),grid_gdf['y'].astype(int)))).toarray()
        else:
            dm = np.zeros(((2 * span + 2),(2 * span + 2)))
        
        ### End added code for deforestaion processing.
        
        ### Begin added code for precip processing.
        w_lat_min = centroid_lat // 0.5 * 0.5
        w_lat_max = w_lat_min + 0.5
        w_lon_min = centroid_lon // 0.625 * 0.625
        w_lon_max = w_lon_min + 0.625
        wt = []
        
        for x in range(-days_back, 1):
            t_date = (centroid_date + dt.timedelta(days = x)).date()

            w1 = np.concatenate([precip_data.iloc[precip_indexer(w_lat_min, w_lon_min, x, t_date, t_year)].values.flatten() for x in range(0,24)])
            w2 = np.concatenate([precip_data.iloc[precip_indexer(w_lat_max, w_lon_min, x, t_date, t_year)].values.flatten() for x in range(0,24)])
            w3 = np.concatenate([precip_data.iloc[precip_indexer(w_lat_min, w_lon_max, x, t_date, t_year)].values.flatten() for x in range(0,24)])
            w4 = np.concatenate([precip_data.iloc[precip_indexer(w_lat_max, w_lon_max, x, t_date, t_year)].values.flatten() for x in range(0,24)])

            wt.append(np.mean([w1,w2,w3,w4], axis = 0))

        wt = np.stack(wt)
        ### End added code for precip processing.

        # Build an array for each day in our date range then stack them. One for the inputs and one for the outputs.
        input_stack = []
        for x in range(-days_back, 1):
            s_df = t_df[t_df['Date'] == (centroid_date + dt.timedelta(days = x))]
            if len(s_df) > 0:
                cm = coo_matrix((s_df['value'],(s_df['lon_grid'].astype(int),s_df['lat_grid'].astype(int)))).toarray()
                cm = np.pad(cm, pad_width=((0, (2 * span + 2) - cm.shape[0]),(0,(2 * span + 2) - cm.shape[1])), mode = 'constant')
            else:
                cm = np.zeros(((2 * span + 2),(2 * span + 2)))
            input_stack.append(cm)
        # New stack logic where we layer fires/defo/wind-U/wind-V and fixed channel ordering.
        input_stack = np.stack([np.stack(input_stack), np.stack([dm]*10)], axis = -1)

        output_stack = []
        for x in range(1, days_forward + 1):
            s_df = t_df[t_df['Date'] == (centroid_date + dt.timedelta(days = x))]
            if len(s_df) > 0:
                cm = coo_matrix((s_df['value'],(s_df['lon_grid'].astype(int),s_df['lat_grid'].astype(int)))).toarray()
                cm = np.pad(cm, pad_width=((0, (2 * span + 2) - cm.shape[0]),(0,(2 * span + 2) - cm.shape[1])), mode = 'constant')
            else:
                cm = np.zeros(((2 * span + 2),(2 * span + 2)))
            output_stack.append(np.stack(cm))
        output_stack = np.stack(output_stack)
        
        ## Save to disk
        np.save(feature_folder_2d + t_index + '.npy', input_stack)
        np.save(feature_folder_1d + t_index + '.npy', wt)
        np.save(label_folder + t_index + '.npy', output_stack)
        
    meta_df.to_csv(meta_folder + 'meta.csv')

### Data import

In [5]:
# Get deforestation data from shapefile.
defo_gpd = gpd.read_file('Yearly_Deforestation_Biome/yearly_deforestation_biome.shp')

# Filter to deforestation shapes in Para.
defo_gpd = defo_gpd[defo_gpd['state'] == 'PA']

# Add bounding boxes to DataFrame.
gpd_bbox = defo_gpd.bounds
defo_gpd['x_min'] = gpd_bbox['minx']
defo_gpd['x_max'] = gpd_bbox['maxx']
defo_gpd['y_min'] = gpd_bbox['miny']
defo_gpd['y_max'] = gpd_bbox['maxy']

# Convert date format.
defo_gpd['Date'] = pd.to_datetime(defo_gpd['image_date'])

In [6]:
# Load VIIRS fire data.
px_combined = pd.read_csv('VIIRS_Para.csv')

In [7]:
# Adjust date format.
px_combined['Date'] = pd.to_datetime(px_combined['YYYYMMDD'].astype(str))

In [8]:
# Add month column so we can sample from September only.
px_combined['Month'] = px_combined['Date'].dt.month

# Add year column so we can segregate train/val/test by year.
px_combined['Year'] = px_combined['Date'].dt.year

In [9]:
# Add a column where every value is a 1 to use in 1-hot encoding strategy.
px_combined['value'] = 1

In [10]:
# Load new precipiation data that also includes wind.
# Samples exist every 0.5 degrees lat and 0.625 degrees lon.
# Warning: this cell requires ~64 GB of RAM to run.
'''
Feature translation for precipation data:
PRECTOTCORR - bias corrected total precipitation
QLML - surface specific humidity
QSH - effective surface specific humidity
QSTAR - surface moisutre scale
RHOA - air density at surface
SPEED - surface wind speed
TLML - surface air temperature
ULML - surface eastward wind
VLML - surface northward wind

Indices: date, time, lat, lon
'''
#precip_data = pd.concat([pd.read_csv('precipitation-data/combined_precip_data_' + str(x) + '.csv') for x in range(2017,2022)])
#precip_data.drop(columns = ['Unnamed: 0','SPEED'], inplace=True)
#precip_data['date'] = pd.to_datetime(precip_data['date'], format='%Y%m%d').dt.date
#precip_data = precip_data.set_index(['date','lat','lon','Time_Hour'])

### Need to break this up by year to improve runtime. Code moved to the end.

'\nFeature translation for precipation data:\nPRECTOTCORR - bias corrected total precipitation\nQLML - surface specific humidity\nQSH - effective surface specific humidity\nQSTAR - surface moisutre scale\nRHOA - air density at surface\nSPEED - surface wind speed\nTLML - surface air temperature\nULML - surface eastward wind\nVLML - surface northward wind\n\nIndices: date, time, lat, lon\n'

### Create datasets (4-Fold Split with precipitation)

In [11]:
%%time
# Build 2017 set.
df_a = px_combined[(px_combined['Month'] > 7) & (px_combined['Month'] < 11) & (px_combined['Year'] == 2017)].sample(n=5000, random_state=321)[['Lon','Lat','Date']]

# Load relevant precipitation data.
precip_data = pd.read_csv('precipitation-data/combined_precip_data_2017.csv')
precip_data.drop(columns = ['Unnamed: 0','SPEED'], inplace=True)
precip_data['date'] = pd.to_datetime(precip_data['date'], format='%Y%m%d').dt.date
precip_data = precip_data.set_index(['date','lat','lon','Time_Hour'])

one_hot_with_defo_precip_stack_and_save(df_a, 
                                 defo_gpd,
                                 precip_data[['PRECTOTCORR', 'QLML', 'QSH', 'QSTAR', 'RHOA', 'TLML','ULML', 'VLML']],
                                 px_combined, 
                                 '4fold_super/2017/', 
                                 '4fold_super/2017/features_2d/', 
                                 '4fold_super/2017/features_1d/', 
                                 '4fold_super/2017/labels/',
                                 span = 15,
                                 days_back = 9,
                                 days_forward = 1
                                 )

# Erase precipiation data to save memory.
precip_data = None

Data Loaded... proceeding with processing
Wall time: 19min 5s


In [12]:
%%time
# Build 2018 set.
df_b = px_combined[(px_combined['Month'] > 7) & (px_combined['Month'] < 11) & (px_combined['Year'] == 2018)].sample(n=5000, random_state=321)[['Lon','Lat','Date']]

# Load relevant precipitation data.
precip_data = pd.read_csv('precipitation-data/combined_precip_data_2018.csv')
precip_data.drop(columns = ['Unnamed: 0','SPEED'], inplace=True)
precip_data['date'] = pd.to_datetime(precip_data['date'], format='%Y%m%d').dt.date
precip_data = precip_data.set_index(['date','lat','lon','Time_Hour'])

one_hot_with_defo_precip_stack_and_save(df_b, 
                                 defo_gpd,
                                 precip_data[['PRECTOTCORR', 'QLML', 'QSH', 'QSTAR', 'RHOA', 'TLML','ULML', 'VLML']],
                                 px_combined,
                                 '4fold_super/2018/', 
                                 '4fold_super/2018/features_2d/', 
                                 '4fold_super/2018/features_1d/',  
                                 '4fold_super/2018/labels/',
                                 span = 15,
                                 days_back = 9,
                                 days_forward = 1
                                 )

# Erase precipiation data to save memory.
precip_data = None

Wall time: 22min 58s


In [13]:
%%time
# Build 2019 set.
df_c = px_combined[(px_combined['Month'] > 7) & (px_combined['Month'] < 11) & (px_combined['Year'] == 2019)].sample(n=5000, random_state=321)[['Lon','Lat','Date']]

# Load relevant precipitation data.
precip_data = pd.read_csv('precipitation-data/combined_precip_data_2019.csv')
precip_data.drop(columns = ['Unnamed: 0','SPEED'], inplace=True)
precip_data['date'] = pd.to_datetime(precip_data['date'], format='%Y%m%d').dt.date
precip_data = precip_data.set_index(['date','lat','lon','Time_Hour'])

one_hot_with_defo_precip_stack_and_save(df_c, 
                                 defo_gpd,
                                 precip_data[['PRECTOTCORR', 'QLML', 'QSH', 'QSTAR', 'RHOA', 'TLML','ULML', 'VLML']],
                                 px_combined, 
                                 '4fold_super/2019/', 
                                 '4fold_super/2019/features_2d/', 
                                 '4fold_super/2019/features_1d/',  
                                 '4fold_super/2019/labels/',
                                 span = 15,
                                 days_back = 9,
                                 days_forward = 1
                                 )

# Erase precipiation data to save memory.
precip_data = None

Wall time: 21min 52s


In [16]:
%%time
# Build 2020 set.
df_d = px_combined[(px_combined['Month'] > 7) & (px_combined['Month'] < 11) & (px_combined['Year'] == 2020)].sample(n=5000, random_state=321)[['Lon','Lat','Date']]

# Load relevant precipitation data.
precip_data = pd.read_csv('precipitation-data/combined_precip_data_2020.csv')
precip_data.drop(columns = ['Unnamed: 0','SPEED'], inplace=True)
precip_data['date'] = pd.to_datetime(precip_data['date'], format='%Y%m%d').dt.date
precip_data = precip_data.set_index(['date','lat','lon','Time_Hour'])

one_hot_with_defo_precip_stack_and_save(df_d, 
                                 defo_gpd,
                                 precip_data[['PRECTOTCORR', 'QLML', 'QSH', 'QSTAR', 'RHOA', 'TLML','ULML', 'VLML']],
                                 px_combined, 
                                 '4fold_super/2020/', 
                                 '4fold_super/2020/features_2d/', 
                                 '4fold_super/2020/features_1d/',  
                                 '4fold_super/2020/labels/',
                                 span = 15,
                                 days_back = 9,
                                 days_forward = 1
                                 )

# Erase precipiation data to save memory.
precip_data = None

Wall time: 23min 4s


In [17]:
%%time
# Build 2021 set.
df_e = px_combined[(px_combined['Month'] > 7) & (px_combined['Month'] < 11) & (px_combined['Year'] == 2021)].sample(n=5000, random_state=321)[['Lon','Lat','Date']]

# Load relevant precipitation data.
precip_data = pd.read_csv('precipitation-data/combined_precip_data_2021.csv')
precip_data.drop(columns = ['Unnamed: 0','SPEED'], inplace=True)
precip_data['date'] = pd.to_datetime(precip_data['date'], format='%Y%m%d').dt.date
precip_data = precip_data.set_index(['date','lat','lon','Time_Hour'])

one_hot_with_defo_precip_stack_and_save(df_e, 
                                 defo_gpd,
                                 precip_data[['PRECTOTCORR', 'QLML', 'QSH', 'QSTAR', 'RHOA', 'TLML','ULML', 'VLML']],
                                 px_combined, 
                                 '4fold_super/2021/', 
                                 '4fold_super/2021/features_2d/', 
                                 '4fold_super/2021/features_1d/',  
                                 '4fold_super/2021/labels/',
                                 span = 15,
                                 days_back = 9,
                                 days_forward = 1
                                 )

# Erase precipiation data to save memory.
precip_data = None

Wall time: 23min 37s
